In [1]:
import json
import math
import copy
import random
import scipy
import os
import time
import numpy as np
from pathlib import Path
import pyproj
import networkx as nx
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as geopd
import shapely
from geopy import distance as geopy_distance
from shapely import ops, LineString
import requests
from itertools import chain
from modules import utils
from networkx import bfs_edges
from modules import generalize
import sys
sys.setrecursionlimit(50000)
print(sys.getrecursionlimit())

50000


In [2]:
import modules.network_extractor as net_ex
from modules.networker import Networker
extractor = net_ex.NetworkExtractor()
extractor.DATA_BASE_PATH = "/home/user/Desktop/JP/street-network-indices/data"
networker = Networker()

In [ ]:
# Extract information from the GHS dataset
urban_centers: geopd.GeoDataFrame

urban_centers = geopd.read_file(
    "/home/user/Desktop/JP/street-network-indices/data/GHS_URBAN_CENTERS/GHS_URBAN_SIMPLIFIED_fixed.gpkg",
    layer='GHS_URBAN_SIMPLIFIED_fixed'    
)
urban_centers = urban_centers.rename(columns={
    "GC_POP_TOT_2025": "population",
    "GC_UCA_KM2_2025": "area",
    "GC_DEV_USR_2025": "continent",
    "GC_UCN_MAI_2025": "name",
    "GC_CNT_GAD_2025": "country"
})
transform = pyproj.Transformer.from_crs("ESRI:54009", "EPSG:4326", always_xy=True).transform

city_name = "quito"
search_name = "Quito"
country = "Ecuador"


# extract info from GHS
city_info = urban_centers.loc[
    (urban_centers["name"] == search_name) &
    (urban_centers["country"] == country)
]

geom = city_info["geometry"].values[0]
geom = ops.transform(transform, geom)

# Load graphs
g_walk = extractor.load_graph(f'{city_name}/graph/walk_{city_name}')
g_bike = extractor.load_graph(f'{city_name}/graph/bike_{city_name}')
g_drive = extractor.load_graph(f'{city_name}/graph/drive_{city_name}')
g_public = extractor.load_graph(f'{city_name}/graph/public_{city_name}')

In [ ]:
# Create small buffer to include potential nodes outside
tiles_buffer = 200 # meters
buffer_distance = tiles_buffer / utils.DEG_CONVERT
buffered = geom.buffer(buffer_distance)

# Get bounds of the polygon
minx, miny, maxx, maxy = buffered.bounds

# Generate grid points
tile_size = 0.005 # degrees
x_coords = np.arange(minx, maxx, tile_size)
y_coords = np.arange(miny, maxy, tile_size)

tiles = {}
i = 0
j = 0
for x in x_coords:
    for y in y_coords:
        tile_geom = shapely.geometry.box(x, y, x + tile_size, y + tile_size)
        if geom.intersects(tile_geom):  # Keep only tiles that intersect original polygon
            tiles[f"{i},{j}"] = {
                "row": i,
                "column": j,
                "min_x": x,
                "max_x": x + tile_size,
                "min_y": y,
                "max_y": y + tile_size,
                "geometry": tile_geom,
                "nodes": [],
                "centroid": tile_geom.centroid
            }
        
        j += 1
    i += 1

for node_id, data in g_walk.nodes(data=True):
    x = data["x"]
    y = data["y"]
    node_tile = list(filter(lambda tile: x >= tile["min_x"] and x <= tile["max_x"] and y >= tile["min_y"] and y <= tile["max_y"], tiles.values()))
    if len(node_tile) != 1:
        # if not found, iterate over all tiles and select the closer 
        min_dist = np.inf
        min_tile = None
        for key,tile in tiles.items():
            dist_to_centroid = utils.distance([x,y], [tile["centroid"].x, tile["centroid"].y])
            if dist_to_centroid < min_dist:
                min_dist = dist_to_centroid
                min_tile = key
        tiles[min_tile]["nodes"].append(node_id)

    else:
        tile = node_tile[0]
        idx = f"{tile['row']},{tile['column']}"
        tiles[idx]["nodes"].append(node_id)

In [43]:
def in_tile(tiles, x, y):
    node_tile = list(filter(lambda tile: x >= tile["min_x"] and x <= tile["max_x"] and y >= tile["min_y"] and y <= tile["max_y"], tiles.values()))
    if len(node_tile) != 1:
        # if not found, iterate over all tiles and select the closer
        min_dist = np.inf
        min_tile = None
        for key,tile in tiles.items():
            dist_to_centroid = utils.distance([x,y], [tile["centroid"].x, tile["centroid"].y])
            if dist_to_centroid < min_dist:
                min_dist = dist_to_centroid
                min_tile = key
        idx = min_tile

    else:
        tile = node_tile[0]
        idx = f"{tile['row']},{tile['column']}"
    return idx

In [44]:
def knn(tiles, x, y, k):
    distances = []
    for key,tile in tiles.items():
        distances.append({
            "key": key,
            "dist": utils.distance([x,y], [tile["centroid"].x, tile["centroid"].y])
        })
    
    distances = sorted(distances, key=lambda d: d['dist'])
    distances = distances[0:k]
    return distances


In [45]:
DEG_CONVERT = 111595.75 
threshold_distance = 300 # meters
buffer_distance = threshold_distance / DEG_CONVERT

# sampled_graph = g_public.subgraph(sampled_nodes)
public_nodes_n = g_public.number_of_nodes()
pubic_nodes_list = list(g_public.nodes())
walk_public = copy.deepcopy(g_walk)
print(f"Finished copy")

first = True
for node_id, data in g_public.nodes(data=True):

    #nearest = knn(tiles, data["x"], data["y"], 2)
    nearest = in_tile(tiles, data["x"], data["y"])
    
    walk_public.add_node(node_id, **data)
    public_point = shapely.Point([data["x"], data["y"]])
    public_point_buffer = utils.buffer_point(public_point, buffer_distance)

    subsample_nodes = tiles[nearest]["nodes"]

    min_dist = np.inf
    min_node = None
    data_xy = [data["x"], data["y"]]
    for walk_node_id in subsample_nodes:
        walk_data = g_walk._node[walk_node_id]

        dist = utils.distance(data_xy, [walk_data["x"], walk_data["y"]])
        if dist < min_dist:
            min_dist = dist
            min_node = walk_node_id
    
    walk_public.add_edge(node_id, min_node, **{"length": dist})

print(f"Finished adding nodes and edges")
all = nx.multi_source_dijkstra_path_length(walk_public, list(g_public.nodes()), weight="length")
mean_len = np.array(list(all.values())).mean()
print(f"Finished calculations")
print(mean_len)
        


Finished copy
Finished adding nodes and edges
Finished calculations
588.9896082078641


In [46]:
extractor.save_as_shp(walk_public, f'{city_name}/shp/walk_public{city_name}')

ValueError: invalid literal for int() with base 10: '5.0'

In [ ]:
# With sampling
# 
DEG_CONVERT = 111595.75 
threshold_distance = 200 # meters
buffer_distance = threshold_distance / DEG_CONVERT

public_nodes_n = g_public.number_of_nodes()
ks = [
    int(public_nodes_n * 0.1), # 10%
    int(public_nodes_n * 0.2), # 20%
    int(public_nodes_n * 0.3), # 30%
    int(public_nodes_n * 0.4), # 40%
    int(public_nodes_n * 0.5), # 50%
    int(public_nodes_n * 0.6), # 60%
    int(public_nodes_n * 0.7), # 70%
    int(public_nodes_n * 0.8), # 80%
    int(public_nodes_n * 0.9), # 90%
]

for k in ks:
    walk_public = copy.deepcopy(g_walk)
    sampled_nodes = random.sample(list(g_public.nodes), k)
    sampled_graph = g_public.subgraph(sampled_nodes)

    print(f"K={k} - starting adding nodes to graph")
    first = True
    for node_id, data in sampled_graph.nodes(data=True):
        walk_public.add_node(node_id, **data)

        public_point = shapely.Point([data["x"], data["y"]])
        public_point_buffer = utils.buffer_point(public_point, buffer_distance)

        for walk_node_id, walk_data in g_walk.nodes(data=True):
            within_distance = shapely.intersects_xy(public_point_buffer, walk_data["x"], walk_data["y"])
            if within_distance:
                dist = utils.distance([data["x"], data["y"]], [walk_data["x"], walk_data["y"]])
                walk_public.add_edge(node_id, walk_node_id, **{"length": dist})
        
        if first: print("finish 1")
        first = False

    print(f"Finished adding nodes")
    
    all = nx.multi_source_dijkstra_path_length(walk_public, sampled_nodes, weight="length")
    mean_len = np.array(list(all.values())).mean()
    print(f"K={k}")
    print(mean_len)
        


In [22]:
values = [
1162.089300796976,
837.4930336142239,
724.5027779501886,
647.0119264220768,
597.7451792200699,
537.6517238468184,
538.1734136584903,
500.2920286385221,
468.17196463057627
]

In [ ]:
plt

In [ ]:

K=150
1162.089300796976

K=301
837.4930336142239

K=452
724.5027779501886

K=603
647.0119264220768

K=754
597.7451792200699

K=905
537.6517238468184

K=1056
538.1734136584903

K=1207
500.2920286385221

K=1358
468.17196463057627

In [31]:
n = g_walk.number_of_nodes()
sampled_nodes = random.sample(list(g_walk.nodes), k)
sampled_graph = g_walk.subgraph(sampled_nodes)

all = nx.multi_source_dijkstra_path_length(g_walk, sampled_nodes, cutoff=20, weight="length")

In [29]:
a = list(all.values())

In [13]:
walk_public = copy.deepcopy(g_walk)

public_nodes_n = g_public.number_of_nodes()
ks = [
    int(public_nodes_n * 0.1), # 10%
    int(public_nodes_n * 0.2), # 20%
    int(public_nodes_n * 0.3), # 30%
    int(public_nodes_n * 0.4), # 40%
    int(public_nodes_n * 0.5), # 50%
    int(public_nodes_n * 0.6), # 60%
    int(public_nodes_n * 0.7), # 70%
    int(public_nodes_n * 0.8), # 80%
    int(public_nodes_n * 0.9), # 90%
]

for k in ks:
    sampled_nodes = random.sample(list(g_public.nodes), k)
    sampled_graph = g_public.subgraph(sampled_nodes)

    for node_id, data in sampled_graph.nodes(data=True):
        print(data)
        break

In [4]:
metrics_result_path = f'{extractor.DATA_BASE_PATH}/results'

def distance(p1, p2):
    return geopy_distance.distance(p1, p2)
    
# conversion from lat/lng to x/y
transform = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform

In [10]:
import scipy.cluster


def generic_stats(input_g: nx.MultiGraph | nx.MultiDiGraph, area: float):
    # metrics for walking
    circuity = 0
    straightness = 0
    euclidean_sum = 0
    network_sum = 0
    edges_len = input_g.number_of_edges()
    elevation_sum = 0
    
    for u,v,data in input_g.edges(data=True):
        p1 = (input_g._node[u]["geometry"].coords[0][1], input_g._node[u]["geometry"].coords[0][0])
        p2 = (input_g._node[v]["geometry"].coords[0][1], input_g._node[v]["geometry"].coords[0][0])
        euclidean_sum += distance(p1,p2).meters

        elevation_sum += data["grade_abs"]
    
        if "length" in data:
            network_sum += data["length"]
        else:
            network_sum += ops.transform(transform, data["geometry"]).length
    
    print("circuity")
    circuity = network_sum / euclidean_sum
    print("straightness")
    straightness = euclidean_sum / network_sum
    print("orientation_entropy")
    orientation_entropy = ox.bearing.orientation_entropy(input_g)
    print("road_density")
    road_density = network_sum / area
    print("average_steepness")
    average_steepness = elevation_sum / edges_len
    print("avg_street_length")
    avg_street_lenght = network_sum / edges_len


    return {
        "circuity": circuity,
        "straightness": straightness,
        "orientation_entropy": orientation_entropy,
        "road_density": road_density,
        "avg_steepness": average_steepness,
        "avg_street_length": avg_street_lenght
    }

def data_has_sidewalk(data):
    if "sidewalk" in data:
        if data["sidewalk"] is not None:
            if data["sidewalk"] == "both" or data["sidewalk"] == "separate" or data["sidewalk"] == "left" or data["sidewalk"] == "right" or data["sidewalk"] == "yes":
                return True
            
    if "sidewalk:both" in data:
        if data["sidewalk:both"] is not None:
            if data["sidewalk:both"] == "separate" or data["sidewalk:both"] == "yes":
                return True

    if "sidewalk:left" in data:
        if data["sidewalk:left"] is not None:
            if data["sidewalk:left"] == "separate" or data["sidewalk:left"] == "yes":
                return True
        
    if "sidewalk:right" in data:
        if data["sidewalk:right"] is not None:
            if data["sidewalk:right"] == "separate" or data["sidewalk:right"] == "yes":
                return True
        
    return False


def calculate_walking_metrics(walk_g: nx.MultiGraph | nx.MultiDiGraph, drive_g: nx.MultiGraph | nx.MultiDiGraph, area: float):
    # generalizer = generalize.Generalize()
    # natural_streets_walking_graph = generalizer.named_streets_generalization(walk_g)
    
    walk_number_of_edges = walk_g.number_of_edges()
    # drive_number_of_edges = drive_g.number_of_edges()
    # walk_drive_ratio = walk_number_of_edges / drive_number_of_edges

    # connectivity = []
    # for node_id, data in natural_streets_walking_graph.nodes(data=True):
    #     edges_len = len(natural_streets_walking_graph[node_id])
    #     connectivity.append(edges_len)

    # connectivity_array = np.array(connectivity)
    # connectivity_mean = np.mean(connectivity_array)
    # connectivity_std = np.std(connectivity_array)
    # connectivity_range = connectivity_array.max() - connectivity_array.min()
    # connectivity_90 = (connectivity_range / 10) * 9
    # connectivity_top_10_len = len(list(filter(lambda x: x > connectivity_90, connectivity_array)))

    # intersection count for undirected graph
    intersections = 0
    for node_id in walk_g.nodes():
        if len(walk_g[node_id]) >= 3:
            intersections += 1

    road_scores_count = 0
    for u,v,data in walk_g.edges(data=True):
        road_score = 0

        road_type = data["highway"]
        try:
            max_speed = int(data["max_speed"])
        except: max_speed = None
        has_sidewalk = data_has_sidewalk(data)
        is_sidewalk = (data["footway"] == "sidewalk") or (road_type == "footway")

        if is_sidewalk or road_type == "path": 
            road_score = 5
        elif "trunk" in road_type and not has_sidewalk:
            road_score = 0
        elif "primary" in road_type and not has_sidewalk:
            road_score = 1
        elif "secondary" in road_type and not has_sidewalk:
            road_score = 2
        elif "tertiary" in road_type:
            road_score = 3
        elif road_type == "residential" or road_type == "living_street":
            road_score = 4
        else: 
            road_score = 3
        
        # Improve rating if it has a sidewalk (not mapped separately)
        if has_sidewalk:
            road_score = (road_score + 2) % 5

        # decrease score depending on road speed
        if max_speed is not None:
            if max_speed >= 30:
                road_score -= 0.5
            elif max_speed >= 50:
                road_score -= 1
            elif max_speed >= 80:
                road_score -= 2
            
            if road_score < 0: road_score = 0

        # sum of road scores
        road_scores_count += road_score


    mean_road_score = road_scores_count / walk_number_of_edges
    # intersection_density = intersections / area
    return {
        # "walk_connectivity_mean": connectivity_mean,
        # "walk_connectivity_std": connectivity_std,
        # "walk_connectivity_top_10p": connectivity_top_10_len
        "walk_connectivity_mean": 0,
        "walk_connectivity_std": 0,
        "walk_connectivity_top_10p": 0,

        "mean_road_score": mean_road_score,
        # "walk_connectivity_mean": connectivity_mean,
        # "walk_connectivity_std": connectivity_std,
        # "walk_connectivity_top_10p": connectivity_top_10_len
        "walk_connectivity_mean": 0,
        "walk_connectivity_std": 0,
        "walk_connectivity_top_10p": 0
    }

In [23]:
def calculate_biking_metrics(bike_g: nx.MultiGraph | nx.MultiDiGraph, drive_g: nx.MultiGraph | nx.MultiDiGraph, area: float):
    # generalizer = generalize.Generalize()
    # natural_streets_bike_graph = generalizer.named_streets_generalization(bike_g)
    # print(natural_streets_bike_graph.number_of_edges())
    # print(natural_streets_bike_graph.number_of_nodes())
    
    bike_number_of_edges = bike_g.number_of_edges()
    # drive_number_of_edges = drive_g.number_of_edges()
    # bike_drive_ratio = bike_number_of_edges / drive_number_of_edges

    # connectivity = []
    # for node_id, data in natural_streets_bike_graph.nodes(data=True):
    #     edges_len = len(natural_streets_bike_graph[node_id])
    #     connectivity.append(edges_len)

    # connectivity_array = np.array(connectivity)
    # connectivity_mean = np.mean(connectivity_array)
    # connectivity_std = np.std(connectivity_array)
    # connectivity_range = connectivity_array.max() - connectivity_array.min()
    # connectivity_90 = (connectivity_range / 10) * 9
    # connectivity_top_10_len = len(list(filter(lambda x: x > connectivity_90, connectivity_array)))

    # intersection count for undirected graph
    # intersections = 0
    # for node_id in bike_g.nodes():
    #     if len(bike_g[node_id]) >= 3:
    #         intersections += 1

    road_scores_count = 0
    for u,v,data in bike_g.edges(data=True):
        road_score = 0
        road_type = data["highway"]
        try:
            max_speed = int(data["max_speed"])
        except: max_speed = None
        if "cycleway" in data:
            cycleway = data["cycleway"]
        else: 
            cycleway = None
        inclination = data["grade_abs"]
        is_cycleway = road_type == "cycleway"

        if is_cycleway: 
            road_score = 5
        elif cycleway is not None:
            if cycleway != "no": #includes lane
                road_score = 4
        elif "trunk" in road_type:
            road_score = 0
        elif "primary" in road_type:
            road_score = 1
        elif "secondary" in road_type:
            road_score = 2
        elif "tertiary" in road_type:
            road_score = 3
        elif road_type == "residential" or road_type == "living_street":
            road_score = 4
        else: 
            road_score = 3

        # decrease score depending on road speed
        if max_speed is not None:
            if max_speed >= 50:
                road_score -= 1
            elif max_speed >= 80:
                road_score -= 2
            
            if road_score < 0: road_score = 0

        # decrease score depending on inclination
        if inclination is not None:
            inclination_percentage = inclination * 100
            if inclination_percentage < 4: #easy inclination
                road_score -= 0
            elif inclination_percentage >= 4 and inclination_percentage < 7: # moderate inclination
                road_score -= 1
            elif inclination_percentage >= 7 and inclination_percentage < 9: # challenging
                road_score -= 2
            elif inclination_percentage > 9: # hard
                road_score -= 3
        
        if road_score < 0: road_score = 0

        # sum of road scores
        road_scores_count += road_score


    mean_road_score = road_scores_count / bike_number_of_edges
    # intersection_density = intersections / area
    return {
        # "bike_intersections_count": intersections,
        # "bike_intersections_density": intersection_density,
        # "bike_drive_ratio": bike_drive_ratio,
        "bike_intersections_count": 0,
        "bike_intersections_density": 0,
        "bike_drive_ratio": 0,
        "mean_road_score": mean_road_score,
        # "bike_connectivity_mean": connectivity_mean,
        # "bike_connectivity_std": connectivity_std,
        # "bike_connectivity_top_10p": connectivity_top_10_len
        "bike_connectivity_mean": 0,
        "bike_connectivity_std": 0,
        "bike_connectivity_top_10p": 0
    }

In [3]:
folder = "ottawa"
# #drive_graph = extractor.load_graph(f"{folder}/graph/{folder}_drive")
# #drive_graph = extractor.load_graph(f"{folder}/graph/{folder}_walk")
#bike_graph = extractor.load_graph(f"{folder}/graph/{folder}_bike")
walk_graph = extractor.load_graph(f"{folder}/graph/{folder}_walk")
extractor.save_as_shp(walk_graph, f"{folder}/shp/{folder}_walk")

In [25]:
# gen = Generalize()#
# small = gen.named_streets_generalization(walk_graph)

In [26]:
def add_result(results_dict, column, value):
    results_dict[column].append(value)

def clean_results(results_dict, max_index):
    for idx in results_dict.keys():
        results_dict[idx] = results_dict[idx][0:max_index]

for folder, city_info in cities.items():

    results_dict = {
        "city": [],
        "area": [],
        "built_up_area": [],
        "population": [],
        "pop_density": [],

        "walk_circuity": [],
        "walk_straightness": [],
        "walk_orientation_entropy": [],
        "walk_road_density": [],
        "walk_avg_steepness": [],
        "walk_avg_street_length": [],
        "walk_intersections_count": [],
        "walk_intersections_density": [],
        "walk_drive_ratio": [],
        "walk_mean_road_score": [],
        "walk_connectivity_mean": [],
        "walk_connectivity_std": [],
        "walk_connectivity_top_10p": [],

        "bike_circuity": [],
        "bike_straightness": [],
        "bike_orientation_entropy": [],
        "bike_road_density": [],
        "bike_avg_steepness": [],
        "bike_avg_street_length": [],
        "bike_intersections_count": [],
        "bike_intersections_density": [],
        "bike_drive_ratio": [],
        "bike_mean_road_score": [],
        "bike_connectivity_mean": [],
        "bike_connectivity_std": [],
        "bike_connectivity_top_10p": []
    }
    results_dict = {
        "city": [],
        "area": [],
        "built_up_area": [],
        "population": [],
        "pop_density": [],

        "walk_mean_road_score": [],

        "bike_mean_road_score": [],
    }

    print(f"start {folder}")
    try:
        total_area = city_stats.loc[city_stats["city"] == city_info["search"]]["area"].values[0]
        area = city_stats.loc[city_stats["city"] == city_info["search"]]["built_up_area"].values[0]
        population = city_stats.loc[city_stats["city"] == city_info["search"]]["population"].values[0]
        population_density = population / area
        add_result(results_dict, "city", folder)
        add_result(results_dict, "area", total_area)
        add_result(results_dict, "built_up_area", area)
        add_result(results_dict, "population", population)
        add_result(results_dict, "pop_density", population_density)
    
        print("loading graphs")
        walk_graph = extractor.load_graph(f"{folder}/graph/{folder}_walk")
        # drive_graph = extractor.load_graph(f"{folder}/graph/{folder}_drive")
        bike_graph = extractor.load_graph(f"{folder}/graph/{folder}_bike")
        print("loaded graphs")

        print("walking metrics")
        # walking_metrics = calculate_walking_metrics(walk_graph, drive_graph, area)
        walking_metrics = calculate_walking_metrics(walk_graph, None, area)
        print("end walking metrics")

        print("biking metrics")
        # biking_metrics = calculate_biking_metrics(bike_graph, drive_graph, area)
        biking_metrics = calculate_biking_metrics(bike_graph, None, area)
        print("end biking metrics")
    
        # print("calculating stats for walking graph")
        # walking_stats = generic_stats(walk_graph, area)
        # print("calculating stats for bike graph")
        # bike_stats = generic_stats(bike_graph, area)
    
        #add_result(results_dict, "walk_circuity", walking_stats["circuity"])
        #add_result(results_dict, "walk_straightness", walking_stats["straightness"])
        #add_result(results_dict, "walk_orientation_entropy", walking_stats["orientation_entropy"])
        #add_result(results_dict, "walk_road_density", walking_stats["road_density"])
        #add_result(results_dict, "walk_avg_steepness", walking_stats["avg_steepness"])
        #add_result(results_dict, "walk_avg_street_length", walking_stats["avg_street_length"])
        #add_result(results_dict, "walk_intersections_count", walking_metrics["walk_intersections_count"])
        #add_result(results_dict, "walk_intersections_density", walking_metrics["walk_intersections_density"])
        #add_result(results_dict, "walk_drive_ratio", walking_metrics["walk_drive_ratio"])
        add_result(results_dict, "walk_mean_road_score", walking_metrics["mean_road_score"])
        #add_result(results_dict, "walk_connectivity_mean", walking_metrics["walk_connectivity_mean"])
        #add_result(results_dict, "walk_connectivity_std", walking_metrics["walk_connectivity_std"])
        #add_result(results_dict, "walk_connectivity_top_10p", walking_metrics["walk_connectivity_top_10p"])

        #add_result(results_dict, "bike_circuity", bike_stats["circuity"])
        #add_result(results_dict, "bike_straightness", bike_stats["straightness"])
        #add_result(results_dict, "bike_orientation_entropy", bike_stats["orientation_entropy"])
        #add_result(results_dict, "bike_road_density", bike_stats["road_density"])
        #add_result(results_dict, "bike_avg_steepness", bike_stats["avg_steepness"])
        #add_result(results_dict, "bike_avg_street_length", bike_stats["avg_street_length"])
        #add_result(results_dict, "bike_intersections_count", biking_metrics["bike_intersections_count"])
        #add_result(results_dict, "bike_intersections_density", biking_metrics["bike_intersections_density"])
        #add_result(results_dict, "bike_drive_ratio", biking_metrics["bike_drive_ratio"])
        add_result(results_dict, "bike_mean_road_score", biking_metrics["mean_road_score"])
        #add_result(results_dict, "bike_connectivity_mean", biking_metrics["bike_connectivity_mean"])
        #add_result(results_dict, "bike_connectivity_std", biking_metrics["bike_connectivity_std"])
        #add_result(results_dict, "bike_connectivity_top_10p", biking_metrics["bike_connectivity_top_10p"])

        results_df = pd.DataFrame(results_dict)
        #results_df.to_csv(f'{metrics_result_path}/results2.csv')
        try:
            results_df_old = pd.read_csv(f'{metrics_result_path}/results_temp.csv', index_col=0)
            results_df_new = pd.concat([results_df_old, results_df]).reset_index(drop=True)
            results_df_new.to_csv(f'{metrics_result_path}/results_temp.csv')
        except:
            results_df.to_csv(f'{metrics_result_path}/results_temp.csv')

        
        


    except Exception as ex:
        print(ex)
        print(f"Error in city {folder}")
    print("-------------------------------------------------------------")
    
results_df_final = pd.read_csv(f'{metrics_result_path}/results_temp.csv', index_col=0)
results_df_final

In [ ]:
results_df_old = pd.read_csv(f'{metrics_result_path}/results_temp.csv', index_col=0).reset_index(drop=True)
results_2 = pd.read_csv(f'{metrics_result_path}/results2.csv', index_col=0).reset_index(drop=True)

results_2["walk_mean_road_score"] = results_df_old["walk_mean_road_score"]
results_2["bike_mean_road_score"] = results_df_old["bike_mean_road_score"]


In [30]:
results_df_new

In [27]:
results_df_old = pd.read_csv(f'{metrics_result_path}/results.csv', index_col=0).reset_index(drop=True)
results_df_old[["city","walk_mean_road_score","bike_mean_road_score"]]

In [18]:
results_df_final[["city", "bike_avg_steepness", "walk_road_density", "bike_avg_street_length", "bike_intersections_count", "bike_intersections_density", "bike_connectivity_mean", "bike_connectivity_top_10p"]]

common_cols = ['city', 'area', 'built_up_area', 'population', 'pop_density']
walk_cols = common_cols + list(filter(lambda x: str(x).find("walk") != -1, results_columns))
bike_cols = common_cols + list(filter(lambda x: str(x).find("bike") != -1, results_columns))

print(walk_cols)
print(bike_cols)

walk_results = results_df_final[walk_cols]
bike_results = results_df_final[bike_cols]

In [19]:
walk_results

In [20]:
bike_results

In [ ]:
ox_results = pd.read_csv(f'/home/geolab/Downloads/indicators.csv')
ox_results.loc[ox_results["core_city"].str.contains("lagos")][["built_up_area","area"]]

In [72]:
stats = pd.read_csv(f'{extractor.DATA_BASE_PATH}/stats.csv', index_col=0)
stats